In [29]:
from urllib.request import Request, urlopen as uReq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import io
import json
import os
import pandas as pd
from config import conn_host, conn_database, conn_user, conn_password
from IPython.display import clear_output
from thefuzz import fuzz
from datetime import datetime as dt

In [30]:
league = "premier-league"
start_season = 2017
end_season = 2023

In [81]:
option = Options()
# option.headless = True
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver.maximize_window()

C:\Users\caiog\AppData\Local\Temp\ipykernel_18648\3468875481.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [89]:
def transform_odds_date(date):
    return dt.strptime(date, '%d.%m.%Y')

seasons = dict()

for season in range(start_season, end_season):
    seasons[season] = []
    
    season_str = f"-{season}-{season+1}" if season != 2022 else ""
    url = f"https://www.betexplorer.com/football/england/{league}{season_str}/results/"
    driver.get(url)

    driver.maximize_window()

    table = driver.find_element(By.XPATH, '//*[@id="js-leagueresults-all"]/div/div/table')
    rows = table.find_elements(By.XPATH, './/tbody/tr')

    total_games = 0
    for i, r in enumerate(rows):
        clear_output()
        print(f"{season}/{end_season-1} {i}/{len(rows)}")
        if not r.text: continue
        tds = r.find_elements(By.XPATH, './/child::td')
        if len(tds) < 6: continue
        matchup, score, home_odds, draw_odds, away_odds, date = [t.text for t in tds]
        
        if not score: continue
        home_score, away_score = score.split(':')
        
        if not matchup: continue
        home_team, away_team = matchup.split(' - ')
        
        
        try:
            if not date.split('.')[-1]:
                date += str(end_season)
                
            match_info = [transform_odds_date(date), home_team, home_score, home_odds, away_team, away_score, away_odds, draw_odds]
            seasons[season].append(match_info)
            total_games += 1
        except Exception as e:
            continue

2022/2022 420/421


In [90]:
driver.close()

In [121]:
def set_fuzz_score(home_team, away_team, row):
    home_score = fuzz.ratio(row["home_team"], home_team)
    away_score = fuzz.ratio(row["away_team"], away_team)
    return home_score + away_score

for season in seasons.keys():
    columns = ['date', 'home_team', 'home_score', 'home_odds', 'away_team', 'away_score', 'away_odds', 'draw_odds']
    odds_df = pd.DataFrame(seasons[season], columns=columns)
    
    season_path = f"./leagues/{league}/{season}-{season+1}.csv"
    season_games = pd.read_csv(season_path, index_col=0)
    season_games["date"] = pd.to_datetime(season_games["date"])
    
    season_games["home_odds"] = None
    season_games["away_odds"] = None
    season_games["draw_odds"] = None
    
    for i, row in season_games.iterrows():
        clear_output()
        print(f"{season}/{end_season} : {i}/{len(season_games)}")
        
        same_date_matches = odds_df[odds_df['date'] == row['date']].reset_index(drop=True)
        same_date_matches['matchup_score'] = same_date_matches.apply(lambda x: set_fuzz_score(row['home_team'], row['away_team'], x), axis=1)
        same_date_matches = same_date_matches.sort_values(by='matchup_score', ascending=False).reset_index(drop=True)
        match = same_date_matches.iloc[0]
        
        season_games.at[i, "home_odds"] = match['home_odds']
        season_games.at[i, "away_odds"] = match['away_odds']
        season_games.at[i, "draw_odds"] = match['draw_odds']
        
    season_games.to_csv(season_path)
        

2022/2023 : 366/367


In [122]:
season_games

,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,home_odds,away_odds,draw_odds
0,2022-08-05,1,Crystal Palace,1.2,0,2,1.0,Arsenal,4.58,1.84,3.57
1,2022-08-06,1,Fulham,1.2,2,2,1.2,Liverpool,10.48,1.28,6.09
2,2022-08-06,1,Tottenham,1.5,4,1,0.5,Southampton,1.36,8.73,5.20
3,2022-08-06,1,Newcastle Utd,1.7,2,0,0.3,Nott'ham Forest,1.58,6.17,4.03
4,2022-08-06,1,Leeds United,0.8,2,1,1.3,Wolves,2.43,3.00,3.35
...,...,...,...,...,...,...,...,...,...,...,...
362,2023-05-20,37,Wolves,1.6,1,1,2.0,Everton,3.14,2.31,3.44
363,2023-05-20,37,Nott'ham Forest,0.6,1,0,0.6,Arsenal,5.18,1.64,4.16
364,2023-05-21,37,West Ham,2.1,3,1,0.9,Leeds United,2.61,2.68,3.46
365,2023-05-21,37,Brighton,2.4,3,1,0.6,Southampton,1.21,12.68,7.30
